In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer # Needed to impute NaN values which exist

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
aqi = pd.read_csv('../input/data.csv', encoding = "ISO-8859-1", parse_dates=['date'], low_memory=False)
aqi.head()

# Dataset
This dataset consists of the following columns:
* **stn_code**: Station Code
* **sampling_date**: Date of sampling (note how this is formatted)
* **state**: State
* **location**: Location of recording
* **agency**: Agency
* **type**: Type of area
* **so2**: Sulphur dioxide (µg/m3)
* **no2**: Nitrogen dioxide (µg/m3)
* **rspm**: Respirable Suspended Particulate Matter (µg/m3)
* **spm**: Suspended Particulate Matter (µg/m3)
* **location_monitoring_station**: Unknown
* **pm2_5**: PSI 2.5 (µg/m3)
* **date**: Date of sampling

In [ ]:
sns.set(style="ticks", rc = {'figure.figsize':(20,15)})

# Supressing update warnings

import warnings
warnings.filterwarnings('ignore')

## Checking the dataset
We can see that there are quite a number of NaNs in the dataset. To proceed with the EDA, we must handle these NaNs by either removing them or filling them. I will be doing both.

In [ ]:
# checking the original dataset
print(aqi.isnull().sum())
print(aqi.shape)
aqi.info()

# Cleaning the dataset
### Removing NaNs
Looking at the dataset head, we can conclude that the following columns:
> - stn_code
> - agency
> - sampling_date
> - location_monitoring_agency 

do not add much to the dataset in terms of information that can't already be extracted from other columns. Therefore, we **drop** these columns.

Since ***date*** also has missing values, we will drop the rows containing these values as they're of little use as well.

### Cleaning values
Since the geographical nomenclature has changed over time, we change it here as well to correspond to more accurate insights.

### The ***type*** column
Currently, the type column has several names for the same type and therefore, it is better to clean it up and make it more uniform. 

In [ ]:
# Cleaning up the data

aqi.drop(['stn_code', 'agency', 'sampling_date', 'location_monitoring_station'], axis=1, inplace=True) #dropping columns that aren't required
aqi = aqi.dropna(subset=['date']) # dropping rows where no date is available

# cleaning up name changes

aqi.state = aqi.state.replace({'Uttaranchal':'Uttarakhand'})
aqi.state[aqi.location == "Jamshedpur"] = aqi.state[aqi.location == 'Jamshedpur'].replace({"Bihar":"Jharkhand"})

#changing types to uniform format

types = {
    "Residential": "R",
    "Residential and others": "RO",
    "Residential, Rural and other Areas": "RRO",
    "Industrial Area": "I",
    "Industrial Areas": "I",
    "Industrial": "I",
    "Sensitive Area": "S",
    "Sensitive Areas": "S",
    "Sensitive": "S",
    np.nan: "RRO"
}
aqi.type = aqi.type.replace(types)

In [ ]:
aqi.head()

#### This looks better with the empty columns dropped. We continue with cleaning the dataset.

In [ ]:
# defining columns of importance, which shall be used reguarly
VALUE_COLS = ['so2', 'no2', 'rspm', 'spm', 'pm2_5']

### Filling NaNs
Since our pollutants column contain a lot of NaNs, we must fill them to have consistent data. If we drop the rows containing NaNs, we will be left with nothing.

I use the `SimpleImputer` from sklearn.imputer (v0.20.2) to fill the missing values in every column with the **mean.**

In [ ]:
# invoking SimpleImputer to fill missing values
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
aqi[VALUE_COLS] = imputer.fit_transform(aqi[VALUE_COLS])

In [ ]:
# checking to see if the dataset has any null values left over and the format
print(aqi.isnull().sum())
aqi.tail()

## Cleaned dataset
Our dataset is now clean with no null values as can be seen. 

*However, doing some EDA locally, I did find that for some states, there were years without any data being collected (2003, for example). I have not imputed the values of these years, which will cause the charts to be broken in between.*

Let's move on to writing functions and doing EDA.

# Exploratory Data Analysis
Make sure to read the comments in the code for better understanding.

## Plotting pollutant levels as yearly averages for states

In [ ]:
# defining a function that plots SO2, NO2, RSPM and SPM yearly average levels for a given state
# since data is available monthly, it was resampled to a year and averaged to obtain yearly averages
# years for which no data was collected has not been imputed
def plot_for_state(state):
    fig, ax = plt.subplots(2,2, figsize=(20,12))
    fig.suptitle(state, size=20)
    state = aqi[aqi.state == state]
    state = state.reset_index().set_index('date')[VALUE_COLS].resample('Y').mean()
    state.so2.plot(legend=True, ax=ax[0][0], title="so2")
    ax[0][0].set_ylabel("so2 (µg/m3)")
    ax[0][0].set_xlabel("Year")
    
    state.no2.plot(legend=True, ax=ax[0][1], title="no2")
    ax[0][1].set_ylabel("no2 (µg/m3)")
    ax[0][1].set_xlabel("Year")
    
    state.rspm.plot(legend=True, ax=ax[1][0], title="rspm")
    ax[1][0].set_ylabel("RSPM (PM10 µg/m3)")
    ax[1][0].set_xlabel("Year")
    
    state.spm.plot(legend=True, ax=ax[1][1], title="spm")
    ax[1][1].set_ylabel("SPM (PM10 µg/m3)")
    ax[1][1].set_xlabel("Year")

In [ ]:
plot_for_state("Uttar Pradesh")

Plotting `Uttar Pradesh`, we see that SO2 levels have fallen in the state while NO2 levels have risen. Information about RSPM and SPM can't be concluded since a lot of data is missing.

## Plotting highest and lowest ranking states


In [ ]:
# defining a function to find and plot the top 10 and bottom 10 states for a given indicator (defaults to SO2)
def top_and_bottom_10_states(indicator="so2"):
    fig, ax = plt.subplots(2,1, figsize=(20, 12))
    
    ind = aqi[[indicator, 'state']].groupby('state', as_index=False).median().sort_values(by=indicator,ascending=False)
    top10 = sns.barplot(x='state', y=indicator, data=ind[:10], ax=ax[0], color='R')
    top10.set_title("Top 10 states by {} (1991-2016)".format(indicator))
    top10.set_ylabel("so2 (µg/m3)")
    top10.set_xlabel("State")
    
    bottom10 = sns.barplot(x='state', y=indicator, data=ind[-10:], ax=ax[1], color='g')
    bottom10.set_title("Bottom 10 states by {} (1991-2016)".format(indicator))
    bottom10.set_ylabel("so2 (µg/m3)")
    bottom10.set_xlabel("State")

In [ ]:
top_and_bottom_10_states("so2")
top_and_bottom_10_states("no2")

Plotting for `SO2`, we can see that the **top** state is `Uttarakhand`, while the **bottom** state is `Meghalaya`.

Plotting for `NO2`, we can see that the **top** state is `West Bengal`, while the **bottom** state is `Mizoram`.

## Plotting the highest ever recorded levels

In [ ]:
# defining a function to find the highest ever recorded levels for a given indicator (defaults to SO2) by state
# sidenote: mostly outliers
def highest_levels_recorded(indicator="so2"):
    plt.figure(figsize=(20,10))
    ind = aqi[[indicator, 'location', 'state', 'date']].groupby('state', as_index=False).max()
    highest = sns.barplot(x='state', y=indicator, data=ind)
    highest.set_title("Highest ever {} levels recorded by state".format(indicator))
    plt.xticks(rotation=90)

In [ ]:
highest_levels_recorded("no2")
highest_levels_recorded("rspm")

Plotting for `NO2`, we can see that `Rajasthan` recorded the highest ever NO2 level.
Plotting for `RSPM`, we can see that `Uttar Pradesh` recorded the highest ever RSPM level.

## Plotting yearly trends

In [ ]:
# defining a function to plot the yearly trend values for a given indicator (defaults to SO2) and state (defaults to overall)
def yearly_trend(state="", indicator="so2", ):
    plt.figure(figsize=(20,12))
    aqi['year'] = aqi.date.dt.year
    if state is "":
        year_wise = aqi[[indicator, 'year', 'state']].groupby('year', as_index=False).median()
        trend = sns.pointplot(x='year', y=indicator, data=year_wise)
        trend.set_title('Yearly trend of {}'.format(indicator))
    else:
        year_wise = aqi[[indicator, 'year', 'state']].groupby(['state','year']).median().loc[state].reset_index()
        trend = sns.pointplot(x='year', y=indicator, data=year_wise)
        trend.set_title('Yearly trend of {} for {}'.format(indicator, state))

In [ ]:
yearly_trend()
yearly_trend("West Bengal", "no2")

Plotting for `SO2`, we can see the yearly trend for sulphur dioxide levels in the country.
Plotting for `NO2` in `West Bengal`, we can see the yearly trend.

## Plotting a heatmap for a particular indicator

In [ ]:
# defining a function to plot a heatmap for yearly median average for a given indicator (defaults to SO2)
def indicator_by_state_and_year(indicator="so2"):
    plt.figure(figsize=(20, 20))
    hmap = sns.heatmap(
        data=aqi.pivot_table(values=indicator, index='state', columns='year', aggfunc='median', margins=True),
               annot=True, linewidths=.5, cbar=True, square=True, cmap='inferno', cbar_kws={'label': "Annual Average"})
    
    hmap.set_title("{} by state and year".format(indicator))

In [ ]:
indicator_by_state_and_year('no2')

## Plotting the relationship between two indicators

In [ ]:
# defining a function to plot the relationship between two indicators on yearly averages.
def relation(ind1="so2", ind2="no2"):
    relation = sns.jointplot(x=ind1, y=ind2, data=aqi.set_index('date').resample('Y').mean(), kind='reg')

In [ ]:
relation()

Plotting for `SO2` and `NO2`, we see a fairly positively sloped line indicating that the two pollutants move simultaneously. 

## Plotting pollutant average by type

In [ ]:
# defining a function to plot pollutant averages by type for a given indicator
def type_avg(indicator=""):
    type_avg = aqi[VALUE_COLS + ['type', 'date']].groupby("type").mean()
    if indicator is not "":
        t = type_avg[indicator].plot(kind='bar')
        plt.xticks(rotation = 0)
        plt.title("Pollutant average by type for {}".format(indicator))
    else:
        t = type_avg.plot(kind='bar')
        plt.xticks(rotation = 0)
        plt.title("Pollutant average by type")

In [ ]:
type_avg('so2')

Plotting for `SO2`, we can see that the `Residential` areas generate most SO2, follwed closely by industrial areas.

## Plotting pollutant averages by locations/state

In [ ]:
# defining a function to plot pollutant averages for a given indicator (defaults to SO2) by locations in a given state
def location_avgs(state, indicator="so2"):
    locs = aqi[VALUE_COLS + ['state', 'location', 'date']].groupby(['state', 'location']).mean()
    state_avgs = locs.loc[state].reset_index()
    sns.barplot(x='location', y=indicator, data=state_avgs)
    plt.title("Location-wise average for {} in {}".format(indicator, state))
    plt.xticks(rotation = 90)

In [ ]:
location_avgs("Uttar Pradesh", "no2")

Plottting for `NO2` and `Uttar Pradesh`, we can see that `Meerut` and `Noida` emitted the most NO2 on average.

I have tried to make the kernel as functional as possible.

# Conclusion
I plan on doing even more EDA on this dataset, such as finding seasonal variations in pollutant levels and others.

### Please leave any feedback you have in the comments section. Thank you!